# 5주차. 신경망 상세 모델링
## 12. 텐서플로를 사용한 사용자 정의 모델과 훈련

# TensorFlow 개요

**텐서플로 제공**  
- 핵심 구조는 넘파이와 매우 비슷하지만 GPU를 지원한다.
- (여러 장치와 서버에 대해서) 분산 컴퓨팅을 지원한다.
- 일종의 JIT 컴파일러를 포함한다.
  - 속도를 높이고 메모리 사용량을 줄이기 위해 계산을 최적화한다.
  - 이를 위해 파이썬 함수에서 계산 그래프를 추출한 다음 최적화하고 효율적으로 실행한다.
- 계산 그래프는 플랫폼에 중립적인 포맥으로 내보낼 수 있으므로 한 환경에서 텐서플로 모델을 훈련하고 다른 환경에서 실행할 수 있다.
- 텐서플로는 자동 미분기능과 RMSProp, Nadam 같은 고성능 옵티마이저를 제공하므로 모든 종류의 손실함수를 쉽게 최소화할 수 있다.

**텐서플로 구조**  
- Operation(op) 단위로 kernel별 구현
- 다양한 OS 지원
- C++, Java, GO, Swift, Javascript 등 Python API 외 다른 언어 지원
  - TensorFlow.js로 브라우저에서 모델 실행 가능
- 광범위한 라이브러리 생태계
  - TensorBoard
  - TFX(TensorFlow Extended)
  - TensorFlow Hub
- 많은 사용자들의 대규모 커뮤니티
  - Stack Overflow 등


# TensorFlow 기본 활용

**텐서와 연산**  
tf.constant() 함수로 텐서를 만들 수 있다.

In [1]:
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.4"

In [2]:
# 두 개의 행과 세 개의 열을 가진 실수 행렬을 나타내는 텐서
tf.constant([[1., 2., 3.], [4., 5., 6.]])

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [3]:
tf.constant(42) # 스칼라

<tf.Tensor: shape=(), dtype=int32, numpy=42>

In [5]:
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])
# ndarray와 마찬가지로 tf.Tensor는 크기(shape)와 데이터 타입(dtype)을 가진다.
t.shape

TensorShape([2, 3])

In [6]:
# 텐서플로에서는 직접 지정하지 않으면 32bit 형태의 자료형을 사용
t.dtype

tf.float32

In [7]:
# 인덱스 참조도 넘파이와 매우 비슷하게 작동
t[:, 1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [8]:
t[..., 1, tf.newaxis]

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

In [9]:
# 모든 종류의 텐서 연산이 가능하다

# 덧셈
t + 10
# tf.add(t, 10)과 같다

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [10]:
# 제곱
tf.square(t)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [11]:
# 행렬 곱셈
t @ tf.transpose(t)
# tf.matmul() 함수를 호출하는 것과 동일하다.

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

**텐서와 넘파이**  
- 넘파이는 기본으로 64비트 정밀도를 사용하지만 텐서플로는 32비트 정밀도를 사용한다.
- 넘파이 배열로 텐서를 만들 때, dtype = tf.float32로 지정해야 한다.

In [13]:
import numpy as np
import os

np.random.seed(42)
tf.random.set_seed(42)

import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

In [14]:
# 텐서는 넘파이와 함께 사용하기 편리하다.
# 넘파이 배열로 텐서를 만들 수 있고 그 반대도 가능하다.
# 넘파이 배열에 텐서플로 연산을 적용할 수 있고 텐서에 넘파이 연산을 적용할 수도 있다.
a = np.array([2., 4., 5.])
tf.constant(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [15]:
t.numpy()

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [17]:
tf.square(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 4., 16., 25.])>

In [16]:
np.square(t)

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

**타입 변환**  
- 타입 변환은 성능을 크게 감소시킬 수 있다.
- 텐서플로는 어떤 타입 변환도 자동으로 수행하지 않는다.
- 호환되지 않는 타입의 텐서로 연산을 실행하면 예외가 발생한다.

In [18]:
# 자료형이 맞지 않는다.
try:
    tf.constant(2.0) + tf.constant(40)
except tf.errors.InvalidArgumentError as ex:
    print(ex)

cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a int32 tensor [Op:AddV2]


In [ ]:
# 비트가 맞지 않는다.
try:
    tf.constant(2.0) + tf.constant(40., dtype=tf.float64)
except tf.errors.InvalidArgumentError as ex:
    print(ex)

**변수**
- 텐서의 내용을 바꿀 수 없기 때문에 일반적인 텐서로는 역전파로 변경되어야 하는 신경망의 가중치를 구현할 수 없다.  
tf.Variable : 변경이 가능한 텐서 객체
- 텐서는 변경이 불가능하지만 tf.Variable을 사용해 변경이 가능하다.

In [19]:
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [21]:
# assign() 메서드를 사용하여 변숫값을 바꿀 수 있다.
# assign_add(), assign_sub() 메서드를 사용하면 주어진 값만큼 변수를 증가시키거나 감소시킬 수 있다.

v.assign(2 * v) # array([[ 2.,  4.,  6.],[ 8., 10., 12.]]

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 4.,  8., 12.],
       [16., 20., 24.]], dtype=float32)>

In [22]:
v[0, 1].assign(42) # array([[ 4., 42., 12.],[16., 20., 24.]]

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 4., 42., 12.],
       [16., 20., 24.]], dtype=float32)>

In [23]:
v[:, 2].assign([0., 1.]) # array([[ 4., 42.,  0.],[16., 20.,  1.]]

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 4., 42.,  0.],
       [16., 20.,  1.]], dtype=float32)>

In [24]:
# 원소의 assign()메서드나 scatter_update(), scatter_nd_update() 메서드를 사용하여 개별 원소를 수정할 수 있다.
v.scatter_nd_update(indices=[[0, 0], [1, 2]],
                    updates=[100., 200.]) # array([[100.,  42.,   0.],[ 16.,  20., 200.]]

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,  42.,   0.],
       [ 16.,  20., 200.]], dtype=float32)>

# 신경망 상세 모델링

**사용자 정의 손실 함수(Custom loss function)**

In [26]:
# 레이블과 예측을 매개변수로 받는 함수를 만들고 텐서플로 연산을 사용해 샘플의 손실을 계산하면 된다.
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss  = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

# 성능을 위해서는 이 예시처럼 벡터화하여 구현해야 한다.
# 또한 텐서플로 그래프의 장점을 활용하려면 텐서플로 연산만 사용해야 한다.

# 전체 손실의 평균이 아니라 샘플마다 하나의 손실을 담은 텐서를 반환하는 것이 좋다.
# 그래야 필요할 때 케라스가 클래스 가중치나 샘플 가중치를 적용할 수 있다.

In [29]:
# 캘리포니아 주택 데이터셋을 로드하여 준비
# 데이터셋을 로드한 다음 훈련 세트, 검증 세트, 테스트 세트로 나눈다.
# 마지막에 스케일을 변경한다.
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

In [30]:
input_shape = X_train.shape[1:]

model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    keras.layers.Dense(1),
])

In [31]:
# 손실을 사용해 케라스 모델의 컴파일 메서드를 호출하고 모델을 훈련할 수 있다.
model.compile(loss=huber_fn, optimizer="nadam", metrics=["mae"])

In [32]:
# 훈련하는 동안 배치마다 케라스는 huber_fn()함수를 호출하여 손실을 계산하고 이를 사용해 경사 하강법을 수행
# 에포크 시작부터 전체 손실을 기록하여 평균 손실을 출력한다.
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 [==============================] - 2s 2ms/step - loss: 0.6235 - mae: 0.9953 - val_loss: 0.2862 - val_mae: 0.5866
Epoch 2/2
363/363 [==============================] - 1s 2ms/step - loss: 0.2197 - mae: 0.5177 - val_loss: 0.2382 - val_mae: 0.5281


**사용자 정의 요소를 가진 모델을 저장하고 로드하기**

In [33]:
model.save("my_model_with_a_custom_loss.h5")

In [34]:
# 사용자 정의 객체를 포함한 모델을 로드할 때는 그 이름과 객체를 매핑해야 한다.
model = keras.models.load_model("my_model_with_a_custom_loss.h5",
                                custom_objects={"huber_fn": huber_fn})

In [35]:
# 매개변수가 있는 함수
def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss  = threshold * tf.abs(error) - threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

In [36]:
model.compile(loss=create_huber(2.0), optimizer="nadam", metrics=["mae"])

In [37]:
model.save("my_model_with_a_custom_loss_threshold_2.h5")

In [38]:
# 새로 정의한 함수 이름이 아니라 저장한 케라스 모델에 사용했던 함수 이름인 "huber_fn"을 사용한다.
model = keras.models.load_model("my_model_with_a_custom_loss_threshold_2.h5",
                                custom_objects={"huber_fn": create_huber(2.0)})

In [39]:
# 모델을 로드할 때 threshold 값을 지정해야 한다.
# keras.losses.Loss 클래스를 상속하고 get_config() 메서드를 구현하여 해결할 수 있다.
class HuberLoss(keras.losses.Loss):
    # 생성자는 기본적인 하이퍼파라미터를 **kwargs로 받은 매개변수 값을 부모 클래스의 생성자에게 전달한다.
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)
    # call() 메서드는 레이블과 예측을 받고 모든 샘플의 손실을 계산하여 반환한다.
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss  = self.threshold * tf.abs(error) - self.threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    # ger_config() 메서드는 하이퍼파라미터 이름과 같이 매핑괸 딕셔너리를 반환한다.
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

In [41]:
# 모델을 컴파일할 때 이 클래스의 인스턴스를 사용할 수 있다.
model.compile(loss=HuberLoss(2.), optimizer="nadam", metrics=["mae"])

In [42]:
model.save("my_model_with_a_custom_loss_class.h5")

In [43]:
# 이 모델을 저장할 때 임곗값도 함께 저장된다.
# 모델을 로드할 때 클래스 이름과 클래스 자체를 매핑해줘야 한다.
model = keras.models.load_model("my_model_with_a_custom_loss_class.h5",
                                custom_objects={"HuberLoss": HuberLoss})

**활성화 함수, 초기화, 규제, 제한을 커스터마이징하기(Customizing)**

In [44]:
# 사용자 정의 활성화 함수
# keras.activations.softplus()나 tf.nn.softplus()와 동일
def my_softplus(z): # tf.nn.softplus(z) 값을 반환합니다
    return tf.math.log(tf.exp(z) + 1.0)

In [45]:
# 사용자 정의 글로럿 초기화
# keras.initializers.glorot_nomal()과 동일
def my_glorot_initializer(shape, dtype=tf.float32):
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)

In [46]:
# 사용자 정의 L1 규제
# keras.regularizers.l1(0.01)과 동일
def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

In [47]:
# 양수인 가중치만 남기는 사용자 정의 제한
# keras.constraints.nonneg()나 tf.nn.relu()와 동일
def my_positive_weights(weights): # tf.nn.relu(weights) 값을 반환합니다
    return tf.where(weights < 0., tf.zeros_like(weights), weights)

In [48]:
# 매개변수는 사용자 정의하려는 함수의 종류에 따라 다르다.
# 만들어진 사용자 정의 함수는 보통의 함수와 동일하게 사용할 수 있다.
layer = keras.layers.Dense(1, activation=my_softplus,
                           kernel_initializer=my_glorot_initializer,
                           kernel_regularizer=my_l1_regularizer,
                           kernel_constraint=my_positive_weights)
# 이 활성화 함수는 Dense 층의 출력에 적용되고 다음 층에 그 결과가 전달된다.
# 층의 가중치는 초기화 함수에서 반환된 값으로 초기화한다.
# 훈련스텝마다 가중치가 규제 함수에 전달되어 규제 손실을 계산하고 전체 손실에 추가되어 훈련을 위한 최종 손실을 만든다.
# 마지막으로 제한 함수가 훈련 스텝마다 호출되어 층의 가중치를 제한한 가중치 값으로 바뀐다.

In [49]:
# factor 하이퍼파라미터를 저장하는 L1규제를 위한 간단한 클래스의 예
class MyL1Regularizer(keras.regularizers.Regularizer):
    def __init__(self, factor):
        self.factor = factor
    def __call__(self, weights):
        return tf.reduce_sum(tf.abs(self.factor * weights))
    def get_config(self):
        return {"factor": self.factor}

**사용자 정의 지표(Custom mertic)**

In [52]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    keras.layers.Dense(1),
])

In [53]:
# Batch 마다 함수 호출하고 epoch 동안의 평균을 기록
model.compile(loss="mse", optimizer="nadam", metrics=[create_huber(2.0)])

In [54]:
# 스트리밍 지표(상태가 있는 지표)
# 단순 평균으로 계산할 수 없으므로 state를 활용

# Precision 클래스 객체를 만들고 이를 함수처럼 사용
precision = keras.metrics.Precision()

# 첫번째 배치와 두번째 배치의 레이블과 예측을 각각 첫 번째 매개변수와 두 번째 매개변수로 전달
precision([0, 1, 1, 1, 0, 1, 0, 1], [1, 1, 0, 1, 0, 1, 0, 1])

<tf.Tensor: shape=(), dtype=float32, numpy=0.8>

In [55]:
precision([0, 1, 0, 0, 1, 0, 1, 1], [1, 0, 1, 1, 0, 0, 0, 0])

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

In [56]:
# result() 메서드를 호출하여 현재 지푯값을 얻을 수 있다.
precision.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

In [58]:
# variables 속성을 사용하여 변수를 확인할 수 있다.
precision.variables

[<tf.Variable 'true_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>,
 <tf.Variable 'false_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>]

In [59]:
# reset_states() 메서드를 사용해 이 변수를 초기화할 수 있다.
precision.reset_states()

In [61]:
# 스트링 지표를 만들고 싶다면 keras.metrics.Metric 클래스 상속
# 전체 후버 손실과 지금까지 처리한 샘플 수를 기록하는 클래스
# 결과값을 요청하면 평균 후버 손실이 반환된다.
class HuberMetric(keras.metrics.Metric):
    def __init__(self, threshold=1.0, **kwargs):
        super().__init__(**kwargs) # 기본 매개변수 처리 (예를 들면, dtype)
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        # 생성자는 add_weight() 메서드를 사용해 여러 배치에 걸쳐 지표의 상태를 기록하기 위한 변수를 만든다.
        self.total = self.add_weight("total", initializer="zeros")
        self.count = self.add_weight("count", initializer="zeros")
    # update_state() 메서드는 이 클래스를 함수처럼 사용할 때 호출된다.
    def update_state(self, y_true, y_pred, sample_weight=None):
        metric = self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(metric))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))
    # result() 메서드는 최종 결과를 계산하고 반환한다.
    def result(self):
        return self.total / self.count
    # get_config() 메서드를 구현하여 threshold 변수를 모델과 함께 저장한다.
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

**사용자 정의 층(Custom layer)**

In [62]:
# Weight가 필요 없는 layer
# 입력에 지수 함수를 적용하는 층
exponential_layer = keras.layers.Lambda(lambda x: tf.exp(x))

In [ ]:
# Dense 층의 간소화 버전을 구현한 것
# weight가 있는 layer
# state와 같이 다룸
class MyDense(keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)
    # buld() 메서드의 역할은 가중치마다 add_weight()메서드를 호출하여 층의 변수를 만드는 것이다.
    def build(self, batch_input_shape):
        self.kernel = self.add_weight(
            name="kernel", shape=[batch_input_shape[-1], self.units],
            initializer="glorot_normal")
        self.bias = self.add_weight(
            name="bias", shape=[self.units], initializer="zeros")
        super().build(batch_input_shape) # must be at the end
    # call() 메서드는 이 층에 필요한 연산을 수행한다.
    def call(self, X):
        return self.activation(X @ self.kernel + self.bias)
    # compute_output_shape() 메서드는 이 층의 출력 크기를 반환한다.
    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape(batch_input_shape.as_list()[:-1] + [self.units])
    # get_config() 메서드는 keras.activations.serialize()를 사용하여 활성화 함수의 전체 설정을 저장
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "units": self.units,
                "activation": keras.activations.serialize(self.activation)}

In [63]:
# 여러 input을 받는 layer
class MyMultiLayer(keras.layers.Layer):
    def call(self, X):
        X1, X2 = X
        print("X1.shape: ", X1.shape ," X2.shape: ", X2.shape) # 사용자 정의 층 디버깅
        return X1 + X2, X1 * X2

    def compute_output_shape(self, batch_input_shape):
        batch_input_shape1, batch_input_shape2 = batch_input_shape
        return [batch_input_shape1, batch_input_shape2]

In [64]:
# Training과 test에서 다르게 동작하는 layer
class AddGaussianNoise(keras.layers.Layer):
    def __init__(self, stddev, **kwargs):
        super().__init__(**kwargs)
        self.stddev = stddev

    def call(self, X, training=None):
        if training:
            noise = tf.random.normal(tf.shape(X), stddev=self.stddev)
            return X + noise
        else:
            return X

    def compute_output_shape(self, batch_input_shape):
        return batch_input_shape

**사용자 정의 모델(Custom model)**

In [65]:
class ResidualBlock(keras.layers.Layer):
    def __init__(self, n_layers, n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(n_neurons, activation="elu",
                                          kernel_initializer="he_normal")
                       for _ in range(n_layers)]

    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        return inputs + Z

In [66]:
class ResidualRegressor(keras.models.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(30, activation="elu",
                                          kernel_initializer="he_normal")
        self.block1 = ResidualBlock(2, 30)
        self.block2 = ResidualBlock(2, 30)
        self.out = keras.layers.Dense(output_dim)

    def call(self, inputs):
        Z = self.hidden1(inputs)
        for _ in range(1 + 3):
            Z = self.block1(Z)
        Z = self.block2(Z)
        return self.out(Z)

**모델 구성 요소에 기반한 손실과 지표**

In [ ]:
# 사용자 정의 재구성 손실을 가지는 모델을 만드는 과정
class ReconstructingRegressor(keras.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        # 생성자가 다섯 개의 은닉층과 하나의 출력층으로 구성된 심층 신경망을 만든다.
        self.hidden = [keras.layers.Dense(30, activation="selu",
                                          kernel_initializer="lecun_normal")
                       for _ in range(5)]
        self.out = keras.layers.Dense(output_dim)
        self.reconstruction_mean = keras.metrics.Mean(name="reconstruction_error")
    # build() 메서드에서 완전 연결 층을 하나 더 추가하여 모델의 입력을 재구성하는 데 사용한다.
    def build(self, batch_input_shape):
        n_inputs = batch_input_shape[-1]
        self.reconstruct = keras.layers.Dense(n_inputs)
        #super().build(batch_input_shape)
    # call() 메서드에서 재구성 손실(재구성과 입력 사이의 평균 제곱 오차)을 계산하고
    # add_loss() 메서드를 사용해 모델의 손실 리스트에 추가한다.
    # 마지막에서 은닉층의 출력을 출력층에 전달하여 얻은 출력값을 반환한다.
    def call(self, inputs, training=None):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        reconstruction = self.reconstruct(Z)
        self.recon_loss = 0.05 * tf.reduce_mean(tf.square(reconstruction - inputs))
        
        if training:
           result = self.reconstruction_mean(recon_loss)
           self.add_metric(result)
        return self.out(Z)

**자동 미분을 사용하여 그레이디언트 계산하기**

In [67]:
# 자동 미분을 사용하여 그레이디언트를 자동으로 계산하는 방법을 이해하기 위해 간단한 함수 예

def f(w1, w2):
    return 3 * w1 ** 2 + 2 * w1 * w2
# w1에 대한 이 함수의 도함수가 6*w1+2*w2이다.
# w2에 대한 도함수는 2*w1이다.

In [68]:
w1, w2 = 5, 3
eps = 1e-6
(f(w1 + eps, w2) - f(w1, w2)) / eps

36.000003007075065

In [69]:
(f(w1, w2 + eps) - f(w1, w2)) / eps

10.000000003174137

In [71]:
# 텐서 플로에서 자동 미분 사용

# 두 변수 w1, w2를 정의
w1, w2 = tf.Variable(5.), tf.Variable(3.)
# tf.GradientTape 블럭을 만들어 이 변수와 관련된 모든 연산을 자동으로 기록
with tf.GradientTape() as tape:
    z = f(w1, w2)

gradients = tape.gradient(z, [w1, w2])

In [72]:
# 텐서플로가 계산한 그레이디언트 확인
gradients

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

In [73]:
# gradient() 메서드가 호출된 후에는 자동으로 테이프가 즉시 지워진다.
# 따라서 gradient() 메서드를 두 번 호출하면 예외가 발생한다.
# gradient() 메서드를 한 번 이상 호출해야 한다면 
# 지속 가능한 테이프를 만들고 사용이 끝난 후 테이프를 삭제하여 리소스를 해제해야 한다.
with tf.GradientTape(persistent=True) as tape:
    z = f(w1, w2)

dz_dw1 = tape.gradient(z, w1)
dz_dw2 = tape.gradient(z, w2) # 작동 이상 없음
del tape

# 텐서플로 함수와 그래프

In [74]:
# 세 제곱을 계산하는 함수
def cube(x) :
  return x ** 3

In [75]:
cube(2)

8

In [76]:
cube(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

In [77]:
# tf.function()을 사용하여 이 파이썬 함수를 텐서플로 함수로 바꾼다
tf_cube = tf.function(cube)
tf_cube

In [78]:
tf_cube(2)

<tf.Tensor: shape=(), dtype=int32, numpy=8>

In [79]:
tf_cube(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

In [80]:
# tf.function 데코레이터
@tf.function
def tf_cube(x) :
  return x ** 3

In [82]:
# 원본 파이썬 함수는 필요할 때 여전히 텐서플로 함수의 python_function 속성으로 참조할 수 있다.
tf_cube.python_function(2)

8

**오토그래프와 트레이싱(AutoGraph와 tracing)**

1. 오토그래프
- 파이썬 함수의 소스 코드를 분석하여 for문, while문, if문, break, continue, return와 같은 제어문을 찾는다.
- 코드를 분석하는 이유는 파이썬이 제어문을 찾을 수 있는 방법을 제공하지 않기 때문이다.
2. 트레이싱
- 트레이싱을 거쳐 tf함수로 변함

**TF function 사용 규칙**
- TF 구성요소가 아닌 다른 라이브러리는 사용하지 않는다.
  - 의도와 다르게 동작하거나 성능 저하를 일으킴
- 반복문보다는 벡터화된 구현을 사용한다.
- 명령을 내린다기보다는 수식을 정의하듯 구현한다.